# Imports

In [1]:
import pandas as pd
import numpy as np

---

In [2]:
# Columnas a cargar
columnas_cargar = ["index", "date", "event_id", "ref_type", "ref_hash", "application_id", "attributed", "device_os_version",
                   "device_brand", "device_model", "device_city", "session_user_agent", "carrier", "kind", "wifi",
                   "connection_type", "ip_address", "device_language", "event_uuid", "trans_id", "user_agent"]

# Definicion de tipos
tipos = {"index" : np.int32,
         "event_id" : np.int16,
         "ref_hash" : np.int64,
         "application_id" : np.int16,
         "attributed" : np.bool,
         "wifi" : np.bool,
         "ip_address" : np.int64}

# Carga de datos
events = pd.read_csv("../../../data/tp2/events.csv", usecols=columnas_cargar, dtype=tipos, parse_dates=["date"])

/opt/anaconda/3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Comentario: Las columnas a las que no se les especifica tipo es porque tienen nulos. Habría que analizar los demás datasets y reencodearlas, de ser posible.

In [3]:
# Ordeno las fechas
events.sort_values(by="date", inplace=True)

In [4]:
#Renombro ref hash
events.rename(columns={"ref_hash" : "device_id"}, inplace=True)

---

## Convierto los ref_tpyes a Android/iOS

In [9]:
events["os_es_android"] = events["ref_type"].replace([1891515180541284343, 1494519392962156891], [1, 0]).astype(np.int8)
# Elimino la columna ref_type
del events["ref_type"]

# Etiquetas temporales

In [10]:
events['hora'] = events['date'].dt.hour
events['dia'] = events['date'].dt.day
events['dia_de_la_semana'] = events['date'].dt.dayofweek

# Franjas horarias

In [11]:
events["hora_2_a_6"] = ((events["hora"] >= 2) & (events["hora"] < 6))
events["hora_6_a_11"] = ((events["hora"] >= 6) & (events["hora"] < 11)) 
events["hora_11_a_14"] = ((events["hora"] >= 11) & (events["hora"] < 14)) 
events["hora_14_a_18"] = ((events["hora"] >= 14) & (events["hora"] < 18)) 
events["hora_18_a_23"] = ((events["hora"] >= 18) & (events["hora"] < 23))
events["hora_23_a_2"] = ((events["hora"] >= 23) | (events["hora"] < 2))

# Armado de ventanas

In [12]:
for ventana_nro in range(1,8):
    events["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= events["date"].dt.day) & ((events["date"].dt.day) <= (19+ventana_nro)))

In [14]:
events.to_pickle("../../../data/tp2/events_tp2_formateado.pkl")